# Introduction
The overall objective of this step is to load data from two CSV files into a "Teams" and "Games" `pandas` dataframe, make sure team names from each dataframe are consistent, and create a new dataframe with features that we can use to train our neural network.  The final dataset should be output as Games-Calculated.csv

## 2.1 Take the Teams and Games CSV files and load them into their own DataFrames.
The original instructions for this Manning liveProject states "Don’t forget to include their column names to make everything easier later on".  However, the CSV files provided in this project do not include column names and we will have to infer what each column means by inspecting the files manually.

The liveProject refers us to *Pandas in Action!* by Boris Paskhaver for how to import data from a CSV file using `pandas`, but no information on how to load static files into Google Colab.  Luckily we can always depend on Stack Overflow for answers to our technical questions.  A good answer on how to work with CSV files (or essentially any file) in Google Colab using a Google Drive account can be found here: [How to read csv to dataframe in Google Colab](https://stackoverflow.com/questions/48340341/how-to-read-csv-to-dataframe-in-google-colab)

We are not given any information about Teams.csv or Games.csv other than they should column names, which they do not.  Here is what we can infer about each file by inspecting them manually:

* Teams.csv: 353 x 2 table with no column headers.
  * Col 1 - Most likely the team conference.
  * Col 2 - Team name.

* Games.csv: 23493 x 5 table with no column headers.
 * Col 1 - A date, most likely when each game was played.
 * Col 2 - A team name (Team 1).
 * Col 3 - An integer, most likely the score of the team in Col 2 (Team 1).
 * Col 4 - A team name (Team 2).
 * Col 5 - An integer, most likely the score of the team in Col 4 (Team 2).

In [ ]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')

teams_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/Data/Teams.csv',
                       header=None,
                       names = ['Conference', 'Team'])

games_df = pd.read_csv('gdrive/My Drive/Colab Notebooks/Data/Games.csv',
                       header=None,
                       names=['Date', 'Team 1', 'Team 1 Score',
                              'Team 2', 'Team 2 Score'],
                       parse_dates=[0])

Mounted at /content/gdrive


In [ ]:
teams_df.head()

,Conference,Team
0,America East,Vermont
1,America East,Stony Brook
2,America East,UMBC
3,America East,Hartford
4,America East,Albany


In [ ]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Conference  353 non-null    object
 1   Team        353 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


In [ ]:
games_df.head()

,Date,Team 1,Team 1 Score,Team 2,Team 2 Score
0,2015-11-13,Hawaii,87,Montana St.,76
1,2015-11-13,Eastern Mich.,70,Vermont,50
2,2015-11-13,Columbia,107,Kean,62
3,2015-11-13,La.-Monroe,88,McMurry,43
4,2015-11-13,Yale,70,Fairfield,57


In [ ]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23493 entries, 0 to 23492
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          23493 non-null  datetime64[ns]
 1   Team 1        23493 non-null  object        
 2   Team 1 Score  23493 non-null  int64         
 3   Team 2        23492 non-null  object        
 4   Team 2 Score  23493 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 917.8+ KB


# 2.1 - Clean the data
We will want to merge these two dataframes together to get conference information from `teams` into `games`, but first we need to make sure the team names used for this merge match between dataframes.

There's a few ways to do this and I'm usually inclined to go with a table join followed by filtering on columns with NULL values when determining values between two columns that do not match.  But, I think in this case it would be easier to think of non-matching values in terms of sets.

We can use set differences to see what values in our `teams` data frame are missing in the Team 1 and Team 2 columns of `games` by converting each column to a set using the `set()` function and subtracting Team 1 and Team 2 of `games` from the Teams column of `teams`.

In [ ]:
teams_not_in_games = set(teams_df["Team"]) - set(games_df["Team 1"]) - set(games_df["Team 2"])
teams_not_in_games

{'Alabama A&M',
 'American University',
 'Appalachian State',
 'Arizona State',
 'Arkansas-Little Rock',
 'Arkansas-Pine Bluff',
 'Boston University',
 'Bryant University',
 'Cal State Bakersfield',
 'Cal State Fullerton',
 'Cal State Northridge',
 'California Baptist',
 'Central Arkansas',
 'Central Connecticut State',
 'Central Michigan',
 'Charleston Southern',
 'Coastal Carolina',
 'Connecticut',
 'East Tennessee State',
 'Eastern Michigan',
 'Eastern Washington',
 'Florida A&M',
 'Florida Gulf Coast',
 'Florida International',
 'Georgia Southern',
 'Grambling State',
 'Illinois-Chicago',
 'Jacksonville State',
 'Kansas State',
 'Long Beach State',
 'Louisiana-Lafayette',
 'Louisiana-Monroe',
 'Loyola (MD)',
 'Maryland-Eastern Shore',
 'Massachusetts-Lowell',
 'McNeese State',
 'Michigan State',
 'Middle Tennessee',
 'Mississippi State',
 'Mississippi Valley State',
 'Montana State',
 "Mount St Mary's",
 'Nebraska-Omaha',
 'New Mexico State',
 'North Carolina A&T',
 'North Carolina

Now we'll combine the sets of Team 1 and Team 2 together and subtract the set of Teams to see what teams show up in the `games` dataframe, but not the `teams` dataframe.

In [ ]:
games_not_in_teams = (set(games_df["Team 1"]) | set(games_df["Team 2"])) - set(teams_df["Team"])
games_not_in_teams

{'A&amp;M-Corpus Chris',
 'Adams St.',
 'Adrian',
 'Alabama A&amp;M',
 'Alabama St.',
 'Alas. Anchorage',
 'Alaska-Anch.',
 'Albany (NY)',
 'Albany St. (GA)',
 'Albion',
 'Alcorn St.',
 'Alice Lloyd',
 'Allen',
 'Alma',
 'American',
 'Anderson (IN)',
 'Anderson (SC)',
 'Angelo St.',
 'Angelo State',
 'Antelope Valley',
 'Appalachian St.',
 'Aquinas',
 'Arcadia',
 'Arizona Christian',
 'Arizona St.',
 'Ark.-Fort Smith',
 'Ark.-Monticello',
 'Ark.-Pine Bluff',
 'Arkansas St.',
 'Arkansas Tech',
 'Arlington Baptist',
 'Army West Point',
 'Asbury',
 'Aub.-Montgomery',
 'Austin',
 'Ave Maria',
 'Averett',
 'Avila',
 'BYU-Hawaii',
 'Bakersfield',
 'Ball St.',
 'Baptist Bible (PA)',
 'Belmont Abbey',
 'Bemidji St.',
 'Bemidji State',
 'Benedictine Mesa',
 'Berea',
 'Bethany (WV)',
 'Bethel (TN)',
 'Bethel (Tenn.)',
 'Bethesda Christian',
 'Biola',
 'Black Hills St.',
 'Bloomsburg',
 'Blue Mountain',
 'Bluefield Col.',
 'Bluefield St.',
 'Bob Jones',
 'Boise St.',
 'Boston U.',
 'Bowie St.',
 

Now for the most unglamorous part of data science, figuring out how to clean some of the data.  What the heck is `A&amp;M-Corpus Chris` supposed to mean?  Well, A &amp M is probably supposed to be A&M, and this team is probably Texas A&M University-Corpus Christi.  Our `teams` dataframe has this listed as Texas A&M-CC.  

Although somewhat labor intensive, we can create a dictionary of team corrections and then apply them to Team 1 and Team 2 of our `games` dataframe with `pandas.DataFrame.replace`.

I'm not a basketball fan, so some of my guesses might be wrong.  But at least we can review our mappings easily in the future with the help of a subject matter expert if there are any questions about how we updated the team names in the `games` dataframe.

In [ ]:
team_corrections = {
    'A&amp;M-Corpus Chris' : 'Texas A&M-CC',
    'Adams St.' : 'Colorado State',
    'Adrian' : 'Michigan',
    'Alabama A&amp;M' : 'Alabama A&M',
    'Alabama St.' : 'Alabama State',
    'Alas. Anchorage',
    'Alaska-Anch.',
    'Albany (NY)',
    'Albany St. (GA)',
    'Albion',
    'Alcorn St.',
    'Alice Lloyd',
    'Allen',
    'Alma',
    'American',
    'Anderson (IN)',
    'Anderson (SC)',
    'Angelo St.',
    'Angelo State',
    'Antelope Valley',
    'Appalachian St.',
    'Aquinas',
    'Arcadia',
    'Arizona Christian',
    'Arizona St.',
    'Ark.-Fort Smith',
    'Ark.-Monticello',
    'Ark.-Pine Bluff',
    'Arkansas St.',
    'Arkansas Tech',
    'Arlington Baptist',
    'Army West Point',
    'Asbury',
    'Aub.-Montgomery',
    'Austin',
    'Ave Maria',
    'Averett',
    'Avila',
    'BYU-Hawaii',
    'Bakersfield',
    'Ball St.',
    'Baptist Bible (PA)',
    'Belmont Abbey',
    'Bemidji St.',
    'Bemidji State',
    'Benedictine Mesa',
    'Berea',
    'Bethany (WV)',
    'Bethel (TN)',
    'Bethel (Tenn.)',
    'Bethesda Christian',
    'Biola',
    'Black Hills St.',
    'Bloomsburg',
    'Blue Mountain',
    'Bluefield Col.',
    'Bluefield St.',
    'Bob Jones',
    'Boise St.',
    'Boston U.',
    'Bowie St.',
    'Boyce',
    'Brescia',
    'Brevard',
    'Brewton Parker',
    'Bridgewater ',
    'Bridgewater (VA)',
    'Bristol',
    'Bryant',
    'Bryn Athyn',
    'Buena Vista',
    'CCNY',
    'CSU Bakersfield',
    'CSU Dom. Hills',
    'CSU Fullerton',
    'CSU Stanislaus',
    'CSUN',
    'Cabrini',
    'Cairn',
    'Cal Baptist',
    'Cal Lutheran',
    'Cal St. Dom. Hills',
    'Cal St. East Bay',
    'Cal St. Fullerton',
    'Cal St. L.A.',
    'Cal St. Monterey Bay',
    'Cal St. Northridge',
    'Cal St. San Marcos',
    'California Merced',
    'Caltech',
    'Campbellsville',
    'Carnegie Mellon',
    'Carroll (MT)',
    'Carver Bible',
    'Catawba',
    'Catholic',
    'Cedarville',
    'Cent. Conn. St.',
    'Centenary (LA)',
    'Centenary (La.)',
    'Centenary (N.J.)',
    'Centenary (NJ)',
    'Central Ark.',
    'Central Baptist',
    'Central Conn. St.',
    'Central Mich.',
    'Central Penn',
    'Central St. (OH)',
    'Central State ',
    'Central Wash.',
    'Chadron St.',
    'Chaminade',
    'Champion Bapt.',
    'Chapman',
    'Charleston So.',
    'Chatham',
    'Chestnut Hill',
    'Cheyney',
    'Chicago St.',
    'Chowan',
    'Cin Clermont',
    'Cincinnati Christian',
    'Claflin',
    'Clarion',
    'Cleveland St.',
    'Coastal Caro.',
    'Coastal Ga.',
    'Coe',
    'Coker',
    'Col. of Charleston',
    'Col. of Idaho',
    'Colo. Christian',
    'Colorado Col.',
    'Colorado Mesa',
    'Colorado St.',
    'Columbia Int&#039;l',
    'Columbia International',
    'Conc. (Texas)',
    'Concord',
    'Concordia (MI)',
    'Concordia (Mich.)',
    'Concordia (NE)',
    'Concordia (Neb.)',
    'Concordia (OR)',
    'Concordia College',
    'Concordia-St. Paul',
    'Coppin St.',
    'Corban',
    'Cornell College',
    'Covenant',
    'Crowley&#039;s Ridge',
    'Crown (MN)',
    'Cumberland ',
    'Cumberland (TN)',
    'Curry',
    'Dakota St.',
    'Dallas',
    'Dalton St.',
    'Daniel Webster',
    'Davis &amp; Elkins',
    'Dean College',
    'Defiance',
    'Delaware St.',
    'Denison',
    'Detroit',
    'Dickinson State',
    'Dillard',
    'Dixie State',
    'Doane',
    'Dominican (Ill.)',
    'Dubuque',
    'E. Texas Bapt.',
    'Earlham',
    'East Central',
    'East Tenn. St.',
    'East Tex. Baptist',
    'East-West U.',
    'East. Mennonite',
    'Eastern',
    'Eastern Ill.',
    'Eastern Ky.',
    'Eastern Mich.',
    'Eastern N.M.',
    'Eastern Ore.',
    'Eastern Oregon',
    'Eastern Wash.',
    'Ecclesia',
    'Eckerd',
    'Edward Waters',
    'Elizabeth City St.',
    'Elmira',
    'Elms',
    'Embry-Riddle (FL)',
    'Emerson',
    'Emmanuel (GA)',
    'Emory',
    'Emporia St.',
    'Erskine',
    'Eureka',
    'FDU-Florham',
    'FGCU',
    'FIU',
    'Fairleigh D&#039;son',
    'Farmingdale St.',
    'Fayetteville St.',
    'Ferris St.',
    'Ferrum',
    'Findlay',
    'Fisher',
    'Fisk',
    'Fla. Atlantic',
    'Fla. Christian',
    'Fla. National',
    'Florida A&amp;M',
    'Florida Col.',
    'Florida Mem.',
    'Florida St.',
    'Florida Tech',
    'Fontbonne',
    'Fort Hays St.',
    'Fort Lewis',
    'Fort Valley St.',
    'Fort Wayne',
    'Francis Marion',
    'Franciscan',
    'Fresno Pacific',
    'Fresno St.',
    'Frostburg St.',
    'Frostburg State',
    'Ga. Southern',
    'Ga. Southwestern',
    'Gallaudet',
    'Geneva',
    'Geo. Washington',
    'George Fox',
    'Georgia College',
    'Georgia St.',
    'Georgian Court',
    'Glenville State',
    'Goldey-Beacom',
    'Gordon',
    'Goshen',
    'Goucher',
    'Grace (IN)',
    'Grace Bible College',
    'Grambling',
    'Great Falls',
    'Greensboro',
    'Hannibal-La Grange',
    'Hardin-Simmons',
    'Harding',
    'Harris-Stowe',
    'Hartwick',
    'Haskell',
    'Hawaii Hilo',
    'Hawaii Pacific',
    'Hawaii-Hilo',
    'Heidelberg',
    'Henderson St.',
    'Hendrix',
    'Hillsdale Baptist',
    'Hiram',
    'Hiwassee',
    'Holy Names',
    'Hood',
    'Howard Payne',
    'Humboldt State',
    'Huntingdon',
    'Huston-Tillotson',
    'IIT',
    'IPFW',
    'Idaho St.',
    'Ill.-Chicago',
    'Ill.-Springfield',
    'Illinois St.',
    'Illinois Tech',
    'Ind.-South Bend',
    'Indiana St.',
    'Indiana-Kokomo',
    'Indiana-Northwest',
    'Indianapolis',
    'Iowa St.',
    'Jackson St.',
    'Jacksonville St.',
    'Jarvis Christian',
    'John Brown',
    'John Jay',
    'John. &amp; Wales (Colo.)',
    'John. &amp; Wales (N.C.)',
    'John. &amp; Wales (R.I.)',
    'Johnson',
    'Johnson &amp; Wales (CO)',
    'Johnson &amp; Wales (NC)',
    'Johnson &amp; Wales (RI)',
    'Judson',
    'Kansas St.',
    'Kean',
    'Kennesaw St.',
    'Kent St.',
    'Kentucky St.',
    'Kentucky Wesleyan',
    'Kenyon',
    'Keystone',
    'Ky. Christian',
    'Ky. Wesleyan',
    'LIFE Pacific',
    'LMU',
    'LSU Shreveport',
    'LSU-Alexandria',
    'La Roche',
    'La Verne',
    'La.-Monroe',
    'LaGrange',
    'Lake Erie',
    'Lakeland',
    'Lamar University',
    'Lander',
    'Le Moyne',
    'LeMoyne-Owen',
    'LeTourneau',
    'Lebanon Valley',
    'Lees-McRae',
    'Lenoir-Rhyne',
    'Lesley',
    'Lewis',
    'Lewis &amp; Clark',
    'Limestone',
    'Lincoln (MO)',
    'Lincoln (PA)',
    'Lincoln (Pa.)',
    'Lindenwood',
    'Linfield',
    'Little Rock',
    'Long Beach St.',
    'Long Island',
    'Louisiana',
    'Louisiana Col.',
    'Louisiana Coll.',
    'Lourdes',
    'Loyola (LA)',
    'Loyola (La.)',
    'Loyola (Md.)',
    'Loyola Chicago',
    'Loyola Maryland',
    'Lynchburg',
    'Lyndon St.',
    'Lyndon State',
    'Lyon',
    'MIT',
    'Madonna',
    'Maine-Fort Kent',
    'Malone',
    'Manchester',
    'Manhattanville',
    'Mansfield',
    'Marian (Ind.)',
    'Mars Hill',
    'Mary',
    'Mary Washington',
    'Marygrove',
    'Maryville (MO)',
    'Marywood',
    'Mass.-Boston',
    'Mass.-Lowell',
    'Master&#039;s',
    'Mayville St.',
    'Mayville State',
    'McGill',
    'McKendree',
    'McMurry',
    'McNeese ',
    'McNeese St.',
    'McPherson',
    'Md.-East. Shore',
    'Me.-Presque Isle',
    'Medaille',
    'Medgar Evers',
    'Menlo',
    'Merchant Marine',
    'Mercy',
    'Messiah',
    'Methodist',
    'Miami (Fla.)',
    'Miami (Ohio)',
    'Mich. St. ',
    'Michigan St.',
    'Michigan-Dearborn',
    'Mid-Atlantic Christ.',
    'Middle Ga. St.',
    'Middle Tenn.',
    'Miles',
    'Millersville',
    'Milligan',
    'Millsaps',
    'Milwaukee Engr.',
    'Minn. Duluth',
    'Minn.-Crookston',
    'Minn.-Morris',
    'Minnesota-Morris',
    'Misericordia',
    'Mississippi',
    'Mississippi Col.',
    'Mississippi Coll.',
    'Mississippi St.',
    'Mississippi Val.',
    'Missouri S&amp;T',
    'Missouri St.',
    'Missouri Valley',
    'Mitchell',
    'Mo. Southern St.',
    'Mo. Western St.',
    'Mo.-St. Louis',
    'Mobile',
    'Molloy',
    'Montana St.',
    'Montana Tech',
    'Montana-Western',
    'Montreat',
    'Moravian',
    'Morehead St.',
    'Morehouse',
    'Morgan St.',
    'Morris',
    'Mount Aloysius',
    'Mt. St. Joseph',
    'Mt. St. Mary (NY)',
    'Mt. St. Mary&#039;s',
    'Mt. St. Vincent',
    'Multnomah Bible',
    'Murray St.',
    'Muskingum',
    'N&#039;west Nazarene',
    'N&#039;western Ohio',
    'N&#039;western St.',
    'N.C. A&amp;T',
    'N.C. Central',
    'N.C. Wesleyan',
    'N.M. Highlands',
    'NC State',
    'NW Christian',
    'NYIT',
    'Nazareth',
    'Neb. Omaha',
    'New Hope Christian',
    'New Mexico St.',
    'Newberry',
    'Newbury',
    'Nicholls St.',
    'No. New Mexico',
    'Norfolk St.',
    'North Carolina St.',
    'North Dakota St.',
    'North Ga.',
    'North Greenville',
    'North Park',
    'Northeastern St.',
    'Northern Ariz.',
    'Northern Colo.',
    'Northern Ill.',
    'Northern Ky.',
    'Northern Mich.',
    'Northland',
    'Northwest (WA)',
    'Northwest (Wash.)',
    'Northwest Indian',
    'Northwest Nazarene',
    'Northwestern Ohio',
    'Northwestern St.',
    'Northwood (MI)',
    'Notre Dame (OH)',
    'Notre Dame (Ohio)',
    'Notre Dame de Namur',
    'Oak Hills Christian',
    'Oakland City',
    'Oakwood',
    'Oberlin',
    'Oglethorpe',
    'Ohio Christian',
    'Ohio Dominican',
    'Ohio Northern',
    'Ohio St.',
    'Ohio St. ',
    'Ohio Valley',
    'Okla. Baptist',
    'Okla. Panhandle',
    'Oklahoma St.',
    'Olivet',
    'Olivet Nazarene',
    'Omaha',
    'Oneonta St.',
    'Oregon St.',
    'Oregon Tech',
    'Ottawa',
    'Ouachita Baptist',
    'Our Lady Of The Lake',
    'Ozarks ',
    'Ozarks (AR)',
    'Pacific Lutheran',
    'Pacific Union',
    'Paine',
    'Palm Beach Atl.',
    'Paul Quinn',
    'Peace',
    'Penn',
    'Penn St.',
    'Penn St.-Harrisburg',
    'Pensacola Christian',
    'Pfeiffer',
    'Phila. Biblical',
    'Piedmont',
    'Piedmont Int&#039;l',
    'Pine Manor',
    'Pitt.-Bradford',
    'Point Loma',
    'Point U.',
    'Pomona-Pitzer',
    'Portland St.',
    'Prairie View',
    'Prairie View A&amp;M',
    'Presentation',
    'Puget Sound',
    'Purdue-Calumet',
    'Purdue-North Cent.',
    'Queens (N.Y.)',
    'Queens (NY)',
    'Quincy',
    'Randolph',
    'Regis (CO)',
    'Reinhardt',
    'Rio Grande',
    'Rivier',
    'Roanoke',
    'Rochester (MI)',
    'Rochester (Mich.)',
    'Rockford',
    'Rockhurst',
    'Rocky Mountain',
    'Rogers St.',
    'Roosevelt',
    'Rosemont',
    'Rowan',
    'Rust',
    'Rutgers-Newark',
    'S&#039;eastern Okla.',
    'S&#039;west Minn. St.',
    'S&#039;western (Texas)',
    'S. Carolina St.',
    'S.C. Upstate',
    'SFA',
    'SIU Edwardsville',
    'SIUE',
    'SUNY Maritime',
    'SUNY Oneonta',
    'Sacramento St.',
    'Saint Joseph&#039;s',
    "Saint Joseph's",
    'Saint Leo',
    'Salem Int&#039;l',
    'Salve Regina',
    'Sam Houston St.',
    'San Diego Christian',
    'San Diego St.',
    'San Fran. St.',
    'San Jose St.',
    'Sarah Lawrence',
    'Savannah St.',
    'Schreiner',
    'Seattle U',
    'Selma',
    'Seton Hill',
    'Sewanee',
    'Shenandoah',
    'Siena Heights',
    'Silver Lake',
    'Simon Fraser',
    'Simpson',
    'Simpson (CA)',
    'Skidmore',
    'Sonoma St.',
    'Sonoma State',
    'South Ala.',
    'South Carolina St.',
    'South Dakota Mines',
    'South Dakota St.',
    'South Fla.',
    'Southeast Mo. St.',
    'Southeastern La.',
    'Southeastern Okla.',
    'Southern California',
    'Southern Ill.',
    'Southern Miss.',
    'Southern Nazarene',
    'Southern Ore.',
    'Southern U.',
    'Southern Univ.',
    'Southern Va.',
    'Southern Wesleyan',
    'Southern-N.O.',
    'Southwestern (TX)',
    'Spalding',
    'Spring Arbor',
    'Spring Hill',
    'St. Ambrose',
    'St. Andrews',
    'St. Bonaventure',
    'St. Catharine',
    'St. Edward&#039;s',
    'St. Francis (B&#039;klyn)',
    'St. Francis (IL)',
    'St. Francis (Ill.)',
    'St. Francis (NY)',
    'St. Francis (PA)',
    'St. Francis (Pa.)',
    'St. John&#039;s ' : "St John's",
    'St. John&#039;s (NY)' : "St John's",
    'St. Jos. (Brkln.)' : "Saint Joseph's (PA)",
    'St. Joseph&#039;s (IN)' : "Saint Joseph's (PA)",
    'St. Martin&#039;s',
    'St. Mary&#039;s (CA)',
    'St. Mary&#039;s (Cal.)',
    'St. Mary&#039;s (MD)',
    'St. Mary&#039;s (MN)',
    'St. Mary&#039;s (Md.)',
    'St. Mary&#039;s (TX)',
    'St. Mary&#039;s (Texas)',
    'St. Peter&#039;s',
    'St. Thomas (MN)',
    'Staten Island',
    'Stephen F. Austin',
    'Stockton',
    'Sul Ross St.',
    'Susquehanna',
    'Tabor',
    'Tampa',
    'Tenn. Wesleyan',
    'Tennessee St.',
    'Tex. A&amp;M Int&#039;l',
    'Tex. A&amp;M-Kingsville',
    'Tex. Permian Basin',
    'Texas A&amp;M',
    'Texas A&amp;M-C.C.',
    'Texas A&amp;M-Comm.',
    'Texas A&amp;M-Int&#039;l',
    'Texas College',
    'Texas Lutheran',
    'Texas St.',
    'Texas Wesleyan',
    'Texas-Dallas',
    'Texas-Tyler',
    'Thiel',
    'Thomas (GA)',
    'Thomas More',
    'Tiffin',
    'Toccoa Falls',
    'Tougaloo',
    'Trevecca Nazarene',
    'Trinity (FL)',
    'Trinity (TX)',
    'Trinity (Texas)',
    'Trinity Baptist',
    'Trinity Christian',
    'Trinity Int&#039;l',
    'Truett-McConnell',
    'Truman',
    'Tusculum',
    'Tuskegee',
    'UALR',
    'UC San Diego',
    'UC Santa Barb.',
    'UC Santa Cruz',
    'UConn',
    'UIC',
    'UMES',
    'UNC Wilmington',
    'UNCG',
    'UNCW',
    'UNI',
    'UT Arlington ',
    'UT Martin',
    'UT Permian Basin',
    'UT Tyler',
    'UTRGV',
    'UVa-Wise',
    'Union (NY)',
    'Union (TN)',
    'University of the Southwest',
    'Urbana',
    'Urbana University',
    'Ursinus',
    'Utah St.',
    'Utica',
    'VCU',
    'VMI',
    'Valley City St.',
    'Valley Forge Chrst.',
    'Vanguard',
    'Voorhees',
    'Wabash',
    'Waldorf',
    'Walla Walla',
    'Warner Southern',
    'Warren Wilson',
    'Wartburg',
    'Wash. &amp; Lee',
    'Wash. Advent.',
    'Washington (Md.)',
    'Washington Adventist',
    'Washington Col.',
    'Washington St.',
    'Wayland Baptist',
    'Wayne St. (NE)',
    'Webber Int&#039;l',
    'Weber St.',
    'Wentworth',
    'Wesley',
    'West Alabama',
    'West Coast Baptist',
    'West Va. Tech',
    'West Va. Wesleyan',
    'Western Caro.',
    'Western Ill.',
    'Western Ky.',
    'Western Mich.',
    'Western N.M.',
    'Western New Eng.',
    'Western Ore.',
    'Western St.',
    'Western State',
    'Westmin. (Pa.)',
    'Westmin. (Utah)',
    'Westminster (MO)',
    'Westminster (PA)',
    'Wheaton (MA)',
    'Wheelock',
    'Whittier',
    'Whitworth',
    'Wichita St.',
    'Widener',
    'Wilberforce',
    'Wiley',
    'Willamette',
    'William &amp; Mary',
    'William Carey',
    'William Jessup',
    'William Jewell',
    'William Woods',
    'Williams Baptist',
    'Wilmington (DE)',
    'Wilmington (Del.)',
    'Wilmington (Ohio)',
    'Wilson',
    'Wis.-La Crosse',
    'Wis.-Parkside',
    'Wis.-Superior',
    'Worcester St.',
    'Wright St.',
    'Xavier (LA)',
    'York (NE)',
    'York (PA)',
    'Youngstown St.',
}